<img src='https://www.actris.eu/sites/default/files/inline-images/Actris%20logo.png' width=200 align=right>

# ACTRIS DC 
## Access ACTRIS Data through the ACTRIS Data Portal

ACTRIS Data Portal: [data.actris.eu](https://data.actris.eu/)

This notebook provides a step-by-step guide on how to access data through the ACTRIS (Aerosols, Clouds, and Trace gases Research Infrastructure) Data Portal. By following this notebook, participants will gain the necessary skills to access and utilize the wealth of atmospheric data available through the ACTRIS Data Portal. This knowledge will enable them to conduct their own research and analysis in the field of atmospheric science. The Data Portal represents a human interface to search and access all ACTRIS data.

Let's get started!

### Import libraries

In [6]:
# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt

# Libraries for working with json files
import json
import requests
import os

### Access the ACTRIS Data Portal, select the desired dataset and download *basket.json* file

You can access the ACTRIS Data Portal [here](https://data.actris.eu/search).

Through the search you can select datasets and add them to your basket by clicking the "Add selection to basket" button. When you have the desired data in your basket, you press the "Download all basket metadata" button. This gives you a *basket.json* file. 

![Screenshots of data search, data selection and basket with "Download all basket metadata" button from ACTRIS Data Portal](https://folk.nilu.no/~lemu/shared_presentations/actris_data_portal_search_select_basket.png)

### Explore downloaded data
This code section is opening a file named "basket.json" located in the "static" directory. If you do not get the code to run, please check that you have the "basket.josn" file in the directory. 

After executing these two lines of code, you can work with the contents of the JSON file using the md variable.

In [3]:
f = open('static/basket.json') # open the file "basket.json"
md = json.load(f) # load the json file

In [4]:
first_element = next(iter(md)) # get the key to the first element of the dictionary
md[first_element] # get the metadata of the first element of the dictionary

{'id': 204487,
 'title': 'OC/EC at Birkenes',
 'matrix': 'aerosol particle phase',
 'instrumentType': 'low volume sampler',
 'station': 'Birkenes I',
 'startTime': '2001-02-06T23:00:00.0000000Z',
 'endTime': '2009-01-06T23:00:00.0000000Z',
 'downloadUrl': 'https://thredds.nilu.no/thredds/fileServer/ebas_doi/QC/2T/UT/QC2T-UT7A.nc',
 'opendapUrl': 'https://thredds.nilu.no/thredds/dodsC/ebas_doi/QC/2T/UT/QC2T-UT7A.nc',
 'variables': ['aerosol particle elemental carbon mass concentration',
  'aerosol particle organic carbon mass concentration',
  'aerosol particle total carbon mass concentration'],
 'pid': 'https://doi.org/10.48597/QC2T-UT7A',
 'repository': 'IN-SITU'}

### Streaming data from basket.json file

Streaming data is done by accessing the 'opendapUrl' metadata element of each file. In this example this shown by only the first element in the basket. 

In [7]:
df_md = pd.DataFrame(md)
df_md.head()


,204487,204488,204489,204493,204495,204497,204498,215239,226175,226177,226179,226227,226229,226231,230419,230430,238273,238275,238277
id,204487,204488,204489,204493,204495,204497,204498,215239,226175,226177,226179,226227,226229,226231,230419,230430,238273,238275,238277
title,OC/EC at Birkenes,OC/EC at Birkenes,OC/EC at Birkenes,OC/EC at Birkenes,OC/EC at Birkenes,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II,OC/EC at Birkenes II
matrix,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase,aerosol particle phase
instrumentType,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler,low volume sampler
station,Birkenes I,Birkenes I,Birkenes I,Birkenes I,Birkenes I,Birkenes II,Birkenes II,Birkenes I,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II,Birkenes II


In [8]:
ds = xr.open_dataset(md[first_element]['opendapUrl'])
ds

<xarray.Dataset>
Dimensions:                             (time: 762, tbnds: 2, metadata_time: 8,
                                         elemental_carbon_pm10_qc_flags: 2,
                                         elemental_carbon_pm25_qc_flags: 2,
                                         organic_carbon_pm10_qc_flags: 2,
                                         organic_carbon_pm25_qc_flags: 2,
                                         total_carbon_pm10_qc_flags: 2,
                                         total_carbon_pm25_qc_flags: 2)
Coordinates:
  * time                                (time) datetime64[ns] 2001-02-09T18:0...
  * metadata_time                       (metadata_time) datetime64[ns] 2001-0...
Dimensions without coordinates: tbnds, elemental_carbon_pm10_qc_flags,
                                elemental_carbon_pm25_qc_flags,
                                organic_carbon_pm10_qc_flags,
                                organic_carbon_pm25_qc_flags,
                                total_carbon_pm10_qc_flags,
                                total_carbon_pm25_qc_flags
Data variables: (12/20)
    time_bnds                           (time, tbnds) datetime64[ns] ...
    metadata_time_bnds                  (metadata_time, tbnds) datetime64[ns] ...
    elemental_carbon_pm10               (time) float64 ...
    elemental_carbon_pm10_qc            (elemental_carbon_pm10_qc_flags, time) float64 ...
    elemental_carbon_pm10_ebasmetadata  (metadata_time) |S64 ...
    elemental_carbon_pm25               (time) float64 ...
    ...                                  ...
    total_carbon_pm10                   (time) float64 ...
    total_carbon_pm10_qc                (total_carbon_pm10_qc_flags, time) float64 ...
    total_carbon_pm10_ebasmetadata      (metadata_time) |S64 ...
    total_carbon_pm25                   (time) float64 ...
    total_carbon_pm25_qc                (total_carbon_pm25_qc_flags, time) float64 ...
    total_carbon_pm25_ebasmetadata      (metadata_time) |S64 ...
Attributes: (12/90)
    Conventions:                   CF-1.8, ACDD-1.3
    featureType:                   timeSeries
    title:                         OC/EC at Birkenes
    keywords:                      mass_concentration_of_organic_carbon_in_pm...
    id:                            QC2T-UT7A.nc
    naming_authority:              EBAS
    ...                            ...
    geospatial_lat_units:          degrees_north
    geospatial_lon_units:          degrees_east
    comment:                       {\n    "Data definition": "EBAS_1.1",\n   ...
    standard_name_vocabulary:      CF-1.7, ACDD-1.3
    history:                       None
    creator_url:                   ebas.nilu.no

In [9]:
ds.data_vars

Data variables:
    time_bnds                           (time, tbnds) datetime64[ns] ...
    metadata_time_bnds                  (metadata_time, tbnds) datetime64[ns] ...
    elemental_carbon_pm10               (time) float64 ...
    elemental_carbon_pm10_qc            (elemental_carbon_pm10_qc_flags, time) float64 ...
    elemental_carbon_pm10_ebasmetadata  (metadata_time) |S64 ...
    elemental_carbon_pm25               (time) float64 ...
    elemental_carbon_pm25_qc            (elemental_carbon_pm25_qc_flags, time) float64 ...
    elemental_carbon_pm25_ebasmetadata  (metadata_time) |S64 ...
    organic_carbon_pm10                 (time) float64 ...
    organic_carbon_pm10_qc              (organic_carbon_pm10_qc_flags, time) float64 ...
    organic_carbon_pm10_ebasmetadata    (metadata_time) |S64 ...
    organic_carbon_pm25                 (time) float64 ...
    organic_carbon_pm25_qc              (organic_carbon_pm25_qc_flags, time) float64 ...
    organic_carbon_pm25_ebasmetadata  

### Download of datasets

In [10]:
# Create the "data_oc_ec" folder if it doesn't exist
if not os.path.exists("data_oc_ec"):
    os.makedirs("data_oc_ec")

for i in md: 
    r = requests.get(md[i]['downloadUrl'])
    filename = r.url[-12:]
    filepath = os.path.join("data_oc_ec", filename)  # Specify the folder path
    with open(filepath, mode="wb") as file:
        file.write(r.content)

In [12]:
ds = xr.open_dataset('static/data_oc_ec/4ZZA-9PJE.nc')
ds

<xarray.Dataset>
Dimensions:                                (time: 6, tbnds: 2,
                                            metadata_time: 1,
                                            elemental_carbon_qc_flags: 1,
                                            organic_carbon_qc_flags: 1,
                                            organic_carbon_corrected_qc_flags: 1,
                                            total_carbon_qc_flags: 1,
                                            total_carbon_corrected_qc_flags: 1)
Coordinates:
  * time                                   (time) datetime64[ns] 2008-09-20T1...
  * metadata_time                          (metadata_time) datetime64[ns] 200...
Dimensions without coordinates: tbnds, elemental_carbon_qc_flags,
                                organic_carbon_qc_flags,
                                organic_carbon_corrected_qc_flags,
                                total_carbon_qc_flags,
                                total_carbon_corrected_qc_flags
Data variables: (12/17)
    time_bnds                              (time, tbnds) datetime64[ns] ...
    metadata_time_bnds                     (metadata_time, tbnds) datetime64[ns] ...
    elemental_carbon                       (time) float64 ...
    elemental_carbon_qc                    (elemental_carbon_qc_flags, time) float64 ...
    elemental_carbon_ebasmetadata          (metadata_time) object ...
    organic_carbon                         (time) float64 ...
    ...                                     ...
    total_carbon                           (time) float64 ...
    total_carbon_qc                        (total_carbon_qc_flags, time) float64 ...
    total_carbon_ebasmetadata              (metadata_time) object ...
    total_carbon_corrected                 (time) float64 ...
    total_carbon_corrected_qc              (total_carbon_corrected_qc_flags, time) float64 ...
    total_carbon_corrected_ebasmetadata    (metadata_time) object ...
Attributes: (12/90)
    Conventions:                   CF-1.8, ACDD-1.3
    featureType:                   timeSeries
    title:                         OC/EC at Birkenes
    keywords:                      mass_concentration_of_organic_carbon_in_pm...
    id:                            4ZZA-9PJE.nc
    naming_authority:              no.nilu.ebas
    ...                            ...
    ebas_framework_contact_name:   Wenche Aas, Kjetil Tørseth, Kjetil Tørseth
    ebas_framework_contact_email:  waa@nilu.no, kt@nilu.no, kt@nilu.no
    ebas_originator:               Yttri, Karl Espen, key@nilu.no, Norwegian ...
    ebas_submitter:                Yttri, Karl Espen, key@nilu.no, Norwegian ...
    ebas_acknowledgement:          Data used in this <study/report/figure/etc...
    ebas_metadata:                 {\n    "Data definition": "EBAS_1.1",\n   ...

To learn how to work with the current datasets, please have a look at "Combining datasets EC OC.ipynb" under Search and access. 